In [4]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from torchvision import transforms
from PIL import Image
import pyttsx3  # For voice feedback

In [6]:

# Load models ONCE outside the loop (critical for stability)
dog_detector = YOLO("yolov8m.pt")
behavior_model = YOLO(r"C:\Users\Abhijeet Singh\Downloads\tensorflow\data_augmented1\Data\runs\detect\train\weights\best.pt")

# Define your behavior classes (MODIFY ACCORDING TO YOUR MODEL)
behavior_classes = ['relax' , 'happy' , 'angry' , 'frown' , 'alert'] 

# Webcam setup with error checking
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # HD width
cap.set(4, 720)   # HD height
if not cap.isOpened():
    raise RuntimeError("Cannot open webcam. Check camera permissions/connections.")

try:
    while True:
        # 1. Safely read frame
        ret, frame = cap.read()
        if not ret:
            print("Frame capture error, restarting...")
            cap.release()
            cap = cv2.VideoCapture(0)
            continue

        # 2. Convert frame for YOLO (BGR to RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 3. Stage 1: Dog detection
        dog_results = dog_detector(rgb_frame, classes=[16], conf=0.4, iou=0.5, verbose=False)
        
        # 4. Process detections
        for box in dog_results[0].boxes.xyxy.cpu().numpy():
            x1, y1, x2, y2 = map(int, box)
            
            # Ensure valid ROI coordinates
            if x1 < x2 and y1 < y2 and all(v >= 0 for v in [x1, y1, x2, y2]):
                try:
                    # 5. Stage 2: Behavior analysis
                    dog_roi = rgb_frame[y1:y2, x1:x2]
                    if dog_roi.size == 0:  # Skip empty ROIs
                        continue
                        
                    behavior_results = behavior_model(dog_roi, verbose=False)
                    
                    # Get behavior label (adapt based on your model type)
                    if behavior_results[0].probs:  # Classification model
                        label = behavior_results[0].probs.top1
                    else:  # Detection model
                        label = int(behavior_results[0].boxes.cls[0])
                    
                    behavior = behavior_classes[label]
                    
                    # Draw results
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, behavior, (x1, y1-10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
                except Exception as e:
                    print(f"ROI processing error: {str(e)}")
                    continue

        # 6. Display output
        cv2.imshow('Dog Behavior Monitor', frame)
        
        # 7. Exit condition
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"Critical error: {str(e)}")
    traceback.print_exc()
finally:
    # Ensure clean resource release
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released safely")

Resources released safely


In [24]:
import cv2
import numpy as np
from collections import deque, defaultdict
import traceback
from ultralytics import YOLO  # Assuming you're using Ultralytics YOLOv8

# Load models once
dog_detector = YOLO("yolov8m.pt")
behavior_model = YOLO(r"C:\Users\Abhijeet Singh\Downloads\tensorflow\data_augmented1\Data\runs\detect\train\weights\best.pt")

behavior_classes = ['relax', 'happy', 'angry', 'frown', 'alert']
CONF_THRESHOLD = 0.5
SMOOTHING_FRAMES = 11  # How many frames to consider for smoothing

# Smoothing dictionary: maps dog_id -> deque of recent labels
dog_label_history = defaultdict(lambda: deque(maxlen=SMOOTHING_FRAMES))

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

if not cap.isOpened():
    raise RuntimeError("Cannot open webcam.")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Frame capture error. Restarting camera.")
            cap.release()
            cap = cv2.VideoCapture(0)
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Stage 1: Detect dogs
        dog_results = dog_detector(rgb_frame, classes=[16], conf=0.4, iou=0.4, verbose=False)

        for i, box in enumerate(dog_results[0].boxes.xyxy.cpu().numpy()):
            x1, y1, x2, y2 = map(int, box)

            if x1 < x2 and y1 < y2 and all(v >= 0 for v in [x1, y1, x2, y2]):
                try:
                    dog_roi = rgb_frame[y1:y2, x1:x2]
                    if dog_roi.size == 0:
                        continue

                    behavior_results = behavior_model(dog_roi, verbose=False)

                    if behavior_results[0].probs:
                        probs = behavior_results[0].probs.data.cpu().numpy()
                        label_idx = int(np.argmax(probs))
                        confidence = float(probs[label_idx])
                    else:
                        boxes = behavior_results[0].boxes
                        if not boxes:
                            continue
                        label_idx = int(boxes.cls[0])
                        confidence = float(boxes.conf[0])

                    if confidence < CONF_THRESHOLD:
                        continue

                    # Smooth predictions using majority vote
                    dog_id = f"{x1}_{y1}_{x2}_{y2}"
                    dog_label_history[dog_id].append(label_idx)
                    most_common_label = max(set(dog_label_history[dog_id]), key=dog_label_history[dog_id].count)
                    behavior = behavior_classes[most_common_label]

                    # Draw bounding box and label
                    label_text = f"{behavior} ({int(confidence * 100)}%)"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label_text, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                except Exception as e:
                    print(f"Error in behavior detection: {e}")
                    continue

        cv2.imshow("Dog Behavior Monitor", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"Critical error: {e}")
    traceback.print_exc()

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released safely.")


Resources released safely.
